In [1]:
import httpx
import requests
import pandas as pd

from enum import Enum
from uuid import UUID
from functools import wraps
from pydantic import BaseModel
from typing import Dict, Union
from duneanalytics import DuneAnalytics

In [2]:
BASE_URL = "https://dune.xyz"
LOGIN_URL = f"{BASE_URL}/auth/login"
API_URL = f"{BASE_URL}/api/"
API_AUTH_URL = f"{API_URL}/auth"
SESSION_URL = f"{API_AUTH_URL}/session"
CSRF_URL = f"{API_AUTH_URL}/csrf"
GRAPH_QL_URL = "https://core-hsr.duneanalytics.com/v1/graphql"

class Verb(str, Enum):
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"
    
    def __contains__(cls, item):
        try:
            cls(item)
        except ValueError:
            return False
        return True    


In [3]:
def decode_if_bytes(text: Union[bytes, str]) -> str:
    if isinstance(text, bytes):
        try:
            text = text.decode('utf-8')
        except UnicodeDecodeError:
            text = text.decode('iso-8859-1')
    return text

def raise_for_status(response: httpx.Response):
    http_error_msg = ''
    reason = decode_if_bytes(response.reason)

    if 400 <= response.status_code < 500:
        http_error_msg = u'%s Client Error: %s for url: %s' % (
            response.status_code, reason, response.url)

    elif 500 <= response.status_code < 600:
        http_error_msg = u'%s Server Error: %s for url: %s' % (
            response.status_code, reason, response.url)

    content: Dict = decode_if_bytes(ast.literal_eval(content))

    if http_error_msg:
        raise httpxx.HTTPError(http_error_msg, response=response)
        

def raise_on_bad_status(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        # Do some checks
        response = func(self, *args, **kwargs)
        if response.status_code < 400:
            return response
        else:
            raise_for_status(response)
    return wrapper



In [4]:
DEFAULT_HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,'
              'image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'dnt': '1',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'origin': BASE_URL,
    'upgrade-insecure-requests': '1'
}

GET_RESULT_QUERY = "query GetResult($query_id: Int!, $parameters: [Parameter!]) {\n  get_result(query_id: $query_id, parameters: $parameters) {\n    job_id\n    result_id\n    __typename\n  }\n}\n"
FIND_RESULT_DATA_BY_RESULT_QUERY = "query FindResultDataByResult($result_id: uuid!) {\n  query_results(where: {id: {_eq: $result_id}}) {\n    id\n    job_id\n    error\n    runtime\n    generated_at\n    columns\n    __typename\n  }\n  get_result_by_result_id(args: {want_result_id: $result_id}) {\n    data\n    __typename\n  }\n}\n"

class Client(httpx.Client):

    @raise_on_bad_status
    def send(self, *args, **kwargs) -> httpx.Response:
        return super().send(*args, **kwargs)
            

class Dune:
    def __init__(self, username: str, password: str):
        self.client = Client()
        self.client.headers.update(DEFAULT_HEADERS)
        self.login(username, password)
    
    def create_graphql_request(operation: str, query: str, variables: dict) -> httpx.Request:
        data = {
            "operationName": operation,
            "query": query_string,
            "variables": variables
        }
        request = self.client.build_request(
            "POST", GRAPH_URL, json=data
        )
        return request
        
        
    def fetch_auth_token(self) -> None:
        response = self.client.post(SESSION_URL)
        token = response.json().get('token')
        self.client.headers.update(
            {'authorization': f'Bearer {token}'}
        )

    
    def login(self, username: str, password: str) -> None:
        self.client.get(LOGIN_URL)
        self.client.post(CSRF_URL)
        csrf_token = self.client.cookies.get('csrf')
        
        form_data = {
            'action': 'login',
            'username': username,
            'password': password,
            'csrf': csrf_token,
            'next': BASE_URL
        }
        
        # Login verification!
        self.client.post(API_AUTH_URL, data=form_data)
        self.fetch_auth_token()
    
    def _post_graph_ql(self, operation: str, query: str, variables: dict) -> dict:
        data = {
            "operationName": operation,
            "query": query,
            "variables": variables
        }
        response = self.client.post(GRAPH_QL_URL, json=data)
        return response.json()
        
    def fetch_raw(self, query_id: int) -> dict:
        result_id_data = self._post_graph_ql(
            "GetResult",
            GET_RESULT_QUERY,
            {"query_id": query_id}
        )
        # TODO investigate this result, maybe break out into seperate function
        result_id = result_id_data\
            .get('data')\
            .get('get_result')\
            .get('result_id')
            
        raw_query_data = self._post_graph_ql(
            "FindResultDataByResult",
            FIND_RESULT_DATA_BY_RESULT_QUERY,
            {"result_id": result_id}
        )

        return raw_query_data
    
    # Should return a DuneQuery Object, that can be used to grab the table, charts etc
    def fetch(self, query_id: int):
        # TODO make an object called DuneQueryResult to encapulate this
        # --> should have name info etc...
        # should raise error
        raw_data = self.fetch_raw(query_id)
        '''
        {'query_results': [{'id': '26c876bb-8030-43f5-bc32-e84a74ac4032',
            'job_id': 'f7c74be8-d75b-4d0b-9d03-acabc0d4dad3',
            'error': None,
            'runtime': 372,
            'generated_at': '2022-03-20T06:56:35.878467+00:00',
            'columns': ['day', 'usd'],
            '__typename': 'query_results'}],

        '''
        # TODO raise on index error?
        columns = raw_data.get('data')\
            .get('query_results')[0]\
            .get('columns')
        
        # Make more efficient?
        results = raw_data.get('data').get('get_result_by_result_id')
        processed = [r['data'] for r in results]
        return pd.DataFrame(processed)
        
        

In [7]:
# TODO, if already logged in, dont log in!!!
# TODO, require env vars
dune = Dune('<USER>', '<PASS>')
data = dune.fetch(108104) #Need to strip links
data.head()

,1D Volume (ETH),1W Volume (ETH),Floor Price,NFT Contract,Owners,Project Name,Rank,Supply,Today's Median Sale,Total Volume ($),Total Volume (ETH)
0,9066.984055,38131.765457,6.491,"<a href=""https://etherscan.io/address/0x7bd294...",6102.0,Meebits,1,20000.0,235.760987,8.351754e+09,2.905506e+06
1,26963.051800,141289.257700,257.280,"<a href=""https://etherscan.io/address/0x4e1f41...",2178.0,Terraforms by Mathcastles,2,9907.0,312.250000,6.385279e+09,2.287316e+06
2,2098.630000,7865.480000,71.000,"<a href=""https://etherscan.io/address/0xb47e3c...",NaN,CryptoPunks,3,NaN,71.475000,2.148868e+09,7.440135e+05
3,26963.051800,141289.257700,257.280,"<a href=""https://etherscan.io/address/0x4e1f41...",2178.0,Terraforms,4,9907.0,312.250000,1.991936e+09,7.168545e+05
4,1.825000,2471.664839,0.150,"<a href=""https://etherscan.io/address/0xce25e6...",2670.0,dotdotdots,5,4870.0,0.150000,1.510427e+09,5.682227e+05


In [13]:
data.dtypes

1D Volume (ETH)        float64
1W Volume (ETH)        float64
Floor Price            float64
NFT Contract            object
Owners                 float64
Project Name            object
Rank                     int64
Supply                 float64
Today's Median Sale    float64
Total Volume ($)       float64
Total Volume (ETH)     float64
dtype: object